# Read Data

In [1]:
import numpy as np
import pandas as pd
import pickle
import json
import csv
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
import numpy as np
import pandas as pd
import datetime as dt
import os

In [2]:
from matplotlib.font_manager import *
myfont = FontProperties(fname='./objs/simhei.ttf', size=14)

In [3]:
def save_obj(obj, name ):
    with open('objs/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('objs/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [6]:
origin_data = pd.read_csv('newdata/infect_city.csv')

In [7]:
origin_data = origin_data.sort_values(by = 'ID', ascending= True)

In [8]:
city_ID = {}
ID_city = {}
for _, row in origin_data.iterrows():
    city_ID[row['city']] = int(row['ID'])
    ID_city[int(row['ID'])] = row['city']

In [9]:
population = origin_data[['ID', 'city', 'population']]
population = np.array(population)

In [11]:
epidemic = origin_data.drop(['population', ], axis= 1)

In [12]:
epidemic = np.array(epidemic)

In [14]:
cure_origin = pd.read_csv('newdata/cure_city.csv')
cure = np.array(cure_origin)
death_origin = pd.read_csv('newdata/death_city.csv')
death = np.array(death_origin)

In [15]:
#来自24个城市的迁入感染人数
iminfect_origin = pd.read_csv('newdata/iminfect.csv')
iminfect = np.array(iminfect_origin)

#城市活跃度
active_origin = pd.read_csv('newdata/huoyue.csv')
active = np.array(active_origin)

alldates = list(origin_data.columns)[3:]

In [18]:
def str_to_dt(datestr):
    return dt.datetime.strptime(datestr, '%Y/%m/%d').date()
def dt_to_str(datedt):
    return datedt.strftime('%Y/%m/%d')

# SICs model

In [83]:
def SI2C(INPUT, t, alpha, k1, k2, k3, k4):
    Y = np.zeros((5))
    alpha = max(alpha, 0)
    k1 = max(k1,0)
    k2 = max(k2,0)
    

    S = INPUT[0]
    I = INPUT[1]
    C = INPUT[2]
    imI = INPUT[3]
    A = INPUT[4]
    
    Y[0] = - k1 * A * I * S
    Y[1] = k1 * A * I * S - alpha * I + k2 * imI
    Y[2] = alpha * I
    Y[3] = k3 * imI
    Y[4] = k4 * A

    return Y

In [84]:
def SIC_im(INPUT, t, alpha, k1, k2, k3):
    Y = np.zeros((4))
    alpha = max(alpha, 0)
    k1 = max(k1,0)
    k2 = max(k2,0)
    
    S = INPUT[0]
    I = INPUT[1]
    C = INPUT[2]
    imI = INPUT[3]
    
    Y[0] = - k1 * I * S
    Y[1] = k1 * I * S - alpha * I + k2 * imI
    Y[2] = alpha * I
    Y[3] = k3 * imI

    return Y

In [85]:
def SIC_act(INPUT, t, alpha, k1, k4):
    Y = np.zeros((4))
    alpha = max(alpha, 0)
    k1 = max(k1,0)
    
    S = INPUT[0]
    I = INPUT[1]
    C = INPUT[2]
    A = INPUT[3]
    
    Y[0] = - k1 * A * I * S
    Y[1] = k1 * A * I * S - alpha * I
    Y[2] = alpha * I
    Y[3] = k4 * A

    return Y

In [86]:
def SIC(INPUT, t, alpha, k1):
    Y = np.zeros((3))
    
    S = INPUT[0]
    I = INPUT[1]
    C = INPUT[2]
    
    Y[0] = - k1 * I * S
    Y[1] = k1 * I * S - alpha * I
    Y[2] = alpha * I

    return Y

In [87]:
def SIR(INPUT, t, alpha, k1):
    Y = np.zeros((3))
    alpha = max(alpha, 0)
    k1 = max(k1,0)

    S = INPUT[0]
    I = INPUT[1]
    R = INPUT[2]
    
    Y[0] = - k1 * I * S
    Y[1] = k1 * I * S - alpha * I
    Y[2] = alpha * I

    return Y

In [88]:
def SEIR(INPUT, t, alpha, gamma, k1):
    Y = np.zeros((4))
    alpha = max(alpha, 0)
    k1 = max(k1,0)

    S = INPUT[0]
    E = INPUT[1]
    I = INPUT[2]
    R = INPUT[3]
    
    Y[0] = - k1 * I * S
    Y[1] = k1 * I * S - alpha * E
    Y[2] = alpha * E - gamma * I
    Y[3] = gamma * I

    return Y

In [89]:
#SI2C
import scipy.integrate as spi
from scipy.optimize import minimize

def SI2C_loss(TRUE, PRED):
    return np.sum(np.square(TRUE[:, 0] - PRED[:, 2])+np.square(TRUE[:, 1] - PRED[:, 3])+np.square(TRUE[:, 2] - PRED[:, 4]))

def optim_fun_si2c(args):
    INPUT, t_range, TRUE = args
    '''
    x[0] = alpha
    x[1] = k1
    x[2] = k2
    x[3] = k3
    x[4] = k4
    '''
    v = lambda x: SI2C_loss(TRUE,spi.odeint(SI2C, (INPUT[0], INPUT[1], INPUT[2], INPUT[3], INPUT[4]), t_range, 
                                      args = (x[0], x[1], x[2], x[3], x[4])))
    return v

In [90]:
import scipy.integrate as spi
from scipy.optimize import minimize

def SIC_im_loss(TRUE, PRED):
    return np.sum(np.square(TRUE[:, 0] - PRED[:, 2])+np.square(TRUE[:, 1] - PRED[:, 3]))

def optim_fun_sic_im(args):
    INPUT, t_range, TRUE = args
    '''
    x[0] = alpha
    x[1] = k1
    x[2] = k2
    x[3] = k3
    '''
    v = lambda x: SIC_im_loss(TRUE,spi.odeint(SIC_im, (INPUT[0], INPUT[1], INPUT[2], INPUT[3]), t_range, 
                                      args = (x[0], x[1], x[2], x[3])))
    return v

In [91]:
import scipy.integrate as spi
from scipy.optimize import minimize

def SIC_act_loss(TRUE, PRED):
    return np.sum(np.square(TRUE[:, 0] - PRED[:, 2])+np.square(TRUE[:, 1] - PRED[:, 3]))

def optim_fun_sic_act(args):
    INPUT, t_range, TRUE = args
    '''
    x[0] = alpha
    x[1] = k1
    x[2] = k4
    '''
    v = lambda x: SIC_act_loss(TRUE,spi.odeint(SIC_act, (INPUT[0], INPUT[1], INPUT[2], INPUT[3]), t_range, 
                                      args = (x[0], x[1], x[2])))
    return v

In [92]:
import scipy.integrate as spi
from scipy.optimize import minimize

def SIC_loss(TRUE, PRED):
    return np.sum(np.square(TRUE[:, 0] - PRED[:, 2]))

def optim_fun_sic(args):
    INPUT, t_range, TRUE = args
    '''
    x[0] = alpha
    x[1] = k1
    '''
    v = lambda x: SIC_loss(TRUE,spi.odeint(SIC, (INPUT[0], INPUT[1], INPUT[2]), t_range, 
                                      args = (x[0], x[1])))
    return v

In [93]:
#SIR
import scipy.integrate as spi
from scipy.optimize import minimize

def SIR_loss(TRUE, PRED):
    return np.sum(np.square(TRUE[:, 0] - TRUE[:,1] -  PRED[:, 1])+np.square(TRUE[:,1]-PRED[:,2]))

def optim_fun_SIR(args):
    INPUT, t_range, TRUE = args
    '''
    x[0] = alpha
    x[1] = k1
    '''
    v = lambda x: SIR_loss(TRUE,spi.odeint(SIR, (INPUT[0], INPUT[1], INPUT[2]), t_range, 
                                      args = (x[0], x[1])))
    return v

In [94]:
#SEIR
import scipy.integrate as spi
from scipy.optimize import minimize

def SEIR_loss(TRUE, PRED):
    return np.sum(np.square(TRUE[:, 0] - TRUE[:,1] -  PRED[:, 2])+np.square(TRUE[:,1]-PRED[:,3]))

def optim_fun_SEIR(args):
    INPUT, t_range, TRUE = args
    '''
    x[0] = alpha
    
    x[1] = gamma
    
    x[2] = k1
    '''
    v = lambda x: SEIR_loss(TRUE,spi.odeint(SEIR, (INPUT[0], INPUT[1], INPUT[2], INPUT[3]), t_range, 
                                      args = (x[0], x[1], x[2])))
    return v

In [97]:
def predict_si2c(init_S,I0coe, confirm_data, iminfect_data, active_data, N, x0_init = None):
    S0 = init_S
    C0 = confirm_data[0]
    I0 = np.sum(confirm_data[0:5])/5
    imI0 = iminfect_data[0]
    A0 = active_data[0]
    INPUT = (S0, I0 * I0coe, C0, imI0, A0)
    
    TRUE = np.array([confirm_data,
                    iminfect_data,
                    active_data])
    TRUE = TRUE.T
    
    t_range = np.arange(0, len(TRUE), 1)
    x0 = x0_init 
    #print('x0',x0)
    RES = minimize(optim_fun_si2c((INPUT, t_range, TRUE)), x0, method = 'Nelder-Mead', options = {'disp': True})

    t_range = np.arange(0.0, N, 1.0)
    x = RES.x
    #print('x',x) 
    RES = spi.odeint(SI2C, (INPUT[0], INPUT[1], INPUT[2], INPUT[3], INPUT[4]), t_range, args = (x[0], x[1], x[2], x[3], x[4]))

    return {'S': RES[:, 0], 'I': RES[:, 1], 'C': RES[:, 2], 'imI': RES[:,3], 'A': RES[:,4]}

In [100]:
def predict_sic_im(init_S,I0coe, confirm_data, iminfect_data, N, x0_init = None):
    S0 = init_S
    C0 = confirm_data[0]
    I0 = np.sum(confirm_data[0:5])/5
    imI0 = iminfect_data[0]
    INPUT = (S0, I0*I0coe, C0, imI0)
    
    TRUE = np.array([confirm_data,
                    iminfect_data])
    TRUE = TRUE.T
    
    t_range = np.arange(0, len(TRUE), 1)
    x0 = x0_init 
    #print('x0',x0)
    RES = minimize(optim_fun_sic_im((INPUT, t_range, TRUE)), x0, method = 'Nelder-Mead', options = {'disp': True})

    t_range = np.arange(0.0, N, 1.0)
    x = RES.x
    #print('x',x) 
    RES = spi.odeint(SIC_im, (INPUT[0], INPUT[1], INPUT[2], INPUT[3]), t_range, args = (x[0], x[1], x[2], x[3]))

    return {'S': RES[:, 0], 'I': RES[:, 1], 'C': RES[:, 2], 'imI': RES[:,3]}

In [101]:
def predict_sic_act(init_S, I0coe, confirm_data, active_data, N, x0_init = None):
    S0 = init_S
    C0 = confirm_data[0]
    A0 = active_data[0]
    I0 = np.sum(confirm_data[0:5])/5
    INPUT = (S0, I0*I0coe, C0, A0)
    
    TRUE = np.array([confirm_data,
                    active_data])
    TRUE = TRUE.T
    
    t_range = np.arange(0, len(TRUE), 1)
    x0 = x0_init 
    #print('x0',x0)
    RES = minimize(optim_fun_sic_act((INPUT, t_range, TRUE)), x0, method = 'Nelder-Mead', options = {'disp': True})

    t_range = np.arange(0.0, N, 1.0)
    x = RES.x
    #print('x',x) 
    RES = spi.odeint(SIC_act, (INPUT[0], INPUT[1], INPUT[2], INPUT[3]), t_range, args = (x[0], x[1], x[2]))

    return {'S': RES[:, 0], 'I': RES[:, 1], 'C': RES[:, 2], 'A': RES[:,3]}

In [102]:
def predict_sic(init_S, I0coe, confirm_data,N, x0_init = None):
    S0 = init_S
    C0 = confirm_data[0]
    I0 = confirm_data[0:5].sum()/5
    INPUT = (S0, I0 * I0coe, C0)
    
    TRUE = np.array([confirm_data,])
    TRUE = TRUE.T
    
    t_range = np.arange(0, len(TRUE), 1)
    x0 = x0_init 
    #print('x0',x0)
    RES = minimize(optim_fun_sic((INPUT, t_range, TRUE)), x0, method = 'Nelder-Mead', options = {'disp': True})

    t_range = np.arange(0.0, N, 1.0)
    x = RES.x
    #print('x',x) 
    RES = spi.odeint(SIC,(INPUT[0], INPUT[1], INPUT[2]), t_range, args = (x[0], x[1]))

    return {'S': RES[:, 0], 'I': RES[:, 1], 'C': RES[:, 2]}

In [103]:
def MAPE(true, pred):
    diff = np.abs(np.array(true) - np.array(pred))
    return np.mean(diff / true)

In [ ]:
def day_add(arr):#计算日新增
    add_arr = np.zeros(len(arr)-1) #新增是从1.17-2.28
    add_arr = arr[1:]-arr[:-1]
    return add_arr

# Grid-search method

In [ ]:
#search the best parameters for si2c
legend_font = {"family":"Times New Roman","size":14}
import cmath
from sklearn.metrics import mean_squared_error
from matplotlib.pyplot import MultipleLocator
from sklearn.model_selection import ParameterGrid
import warnings
import csv
warnings.filterwarnings("ignore")

foutput = open('metric/si2c_results_1.csv','w')
foutput2 = open('metric/si2c_bestresults_1.csv','w')
csv_writer = csv.writer(foutput)
csv_writer_best = csv.writer(foutput2)

predictors = {} 
for cityname in [
'北京市',
"上海市",
"广州市",
"温州市",
"深圳市",
"武汉市",
"黄冈市",
"孝感市",
"天门市",
"南昌市",
"长沙市",
"恩施市",
"信阳市",
"仙桃市",
"重庆市",
"十堰市",
"咸宁市",
"荆门市",
"宜昌市",
"黄石市",
"襄阳市",
"随州市",
"鄂州市",
"荆州市"
]:
    cityid = city_ID[cityname]
    init_S = population[cityid, 2] * 10000
    window = 44
    for i in range(len(epidemic[cityid])):
        if(epidemic[cityid,-(i+1)]==0):
            window = i
            break
    print('================='+cityname+'===================')
    print('window',window)
    confirm_data = epidemic[cityid, -window:]
    active_data = active[cityid, -window:]
    iminfect_data = iminfect[cityid, -window:]
    
    RMSE_ALL = []
    MAPE_ALL = []
    param_grid = {'k1': np.arange(0.1, 1.7, 0.1),
                  'k2': np.arange(2e-8, 3.2e-8, 0.2e-8), 
                  'I0': np.arange(0.1, 2, 0.05)}
    NN=240

    param_name = ['k1','k2', 'I0']
    Grid = ParameterGrid(param_grid)
    epoch = 0 
    for grid in Grid:
        args = [grid[name] for name in param_name]
        args_new = args[0:2]
        args_new.extend([20,0,0])
        print(epoch, args_new, args[-1])
        data_all = predict_si2c(init_S, args[-1], confirm_data, iminfect_data, active_data, NN, x0_init=args_new)
        confirm_all = data_all['C']
        infect_all = data_all['I']
        rmset = np.sqrt(mean_squared_error(confirm_data,confirm_all[:len(confirm_data)]))
        mapent = MAPE(confirm_data,confirm_all[:len(confirm_data)])
        RMSE_ALL.append(rmset)
        MAPE_ALL.append(mapent)
        
        writelist = [statename]
        writelist.append(epoch)
        for arg in args_new:
            writelist.append(arg)
        writelist.append(args[-1])
        writelist.append(rmset)
        writelist.append(mapent)
        csv_writer.writerow(writelist)
           
        epoch = epoch + 1
    
    minrmse = RMSE_ALL[0]
    minind = 0
    for i, rmse in enumerate(RMSE_ALL):
        if rmse < minrmse:
            minrmse = rmse
            minind = i
    print(RMSE_ALL)
    print(MAPE_ALL)
    print(minrmse, MAPE_ALL[minind], minind)
    csv_writer_best.writerow([statename, minrmse, MAPE_ALL[minind], minind])
    

foutput.close()
foutput2.close()

In [70]:
#SIC-Act参数
params = {}
# alpha:感染者转化为确诊者的概率
# k1：活跃度系数*传染率/N
# k4：活跃变化系数

params['武汉市'] = np.array([0.3, 3e-8,  0])#34
params["天门市"] = np.array([0.3, 2.9e-8,  0])

params['黄冈市'] = np.array([0.3, 2.9e-8,  0])#win=33,倍数=2
params['深圳市'] = np.array([0.3, 3.4e-8,  0]) #win=31 ,倍数=2  ##30
params['广州市'] = np.array([0.3, 3e-8, 0])#win=35 ##43
params['孝感市'] = np.array([0.3, 2.9e-8,  0])#win=34
params['温州市'] = np.array([0.3, 2.9e-8,  0])#35
params['北京市'] = np.array([0.3, 2.9e-8,  0])#30
params['上海市'] = np.array([0.3, 3e-8,  0])#37  ##42
params["长沙市"] = np.array([0.3, 2.9e-8,  0])
params["恩施市"] = np.array([0.3, 2.9e-8,  0])
params["信阳市"] = np.array([0.3, 3e-8,  0])
params["仙桃市"] = np.array([0.3, 2.9e-8,  0])
params["重庆市"] = np.array([0.3, 2.9e-8,  0])
params["十堰市"] = np.array([0.3, 2.9e-8,  0])
params["咸宁市"] = np.array([0.3, 2.9e-8,  0])
params["荆门市"] = np.array([0.3, 2.9e-8,  0])
params["宜昌市"] = np.array([0.3, 2.9e-8,  0])
params["黄石市"] = np.array([0.3, 2.9e-8,  0])
params["襄阳市"] = np.array([0.3, 2.9e-8,  0])
params["随州市"] = np.array([0.3, 2.9e-8,  0])
params["鄂州市"] = np.array([0.3, 2.9e-8,  0])
params["荆州市"] = np.array([0.3, 2.9e-8,  0])
params["南昌市"] = np.array([0.3, 2.9e-8,  0])

legend_font = {"family":"Times New Roman","size":14}
import cmath
from sklearn.metrics import mean_squared_error
from matplotlib.pyplot import MultipleLocator
predictors = {} 
for cityname in [
'北京市',
"上海市",
"广州市",
"温州市",
"深圳市",
"武汉市",
"黄冈市",
"孝感市",
"天门市",
"南昌市",
"长沙市",
"恩施市",
"信阳市",
"仙桃市",
"重庆市",
"十堰市",
"咸宁市",
"荆门市",
"宜昌市",
"黄石市",
"襄阳市",
"随州市",
"鄂州市",
"荆州市"
]:
    cityid = city_ID[cityname]
    init_S = population[cityid, 2] * 10000
    window = 44
    for i in range(len(epidemic[cityid])):
        if(epidemic[cityid,-(i+1)]==0):
            window = i
            break

    confirm_data = epidemic[cityid, -window:]
    iminfect_data = iminfect[cityid, -window:]
    active_data = active[cityid, -window:]
    try:
        x0 = params[cityname]
    except:
        x0 = None

    NN = 240
    data_all = predict_sic_act(init_S, confirm_data, active_data, NN, confirm_data[0], x0_init=x0)
    confirm_all = data_all['C']
    infect_all = data_all['I']
    confirm = confirm_all[:len(confirm_data)+6]
    infect = infect_all[:len(confirm_data)+6]
    
    data_all2 = predict_sic_act(init_S, confirm_data, active_data, NN, x0_init=x0)
    confirm_all2 = data_all2['C']
    infect_all2 = data_all2['I']
    confirm2 = confirm_all2[:len(confirm_data)+6]
    infect2 = infect_all2[:len(confirm_data)+6]

    data_all3 = predict_sic_act(init_S, confirm_data, active_data, NN,confirm_data[0]*3, x0_init=x0)
    confirm_all3 = data_all3['C']
    infect_all3 = data_all3['I']
    confirm3 = confirm_all3[:len(confirm_data)+6]
    infect3 = infect_all3[:len(confirm_data)+6]
    
    based_time = str_to_dt(alldates[-window])
    t_range_dt = [(based_time + (dt.timedelta(days=x))).strftime("%m/%d") for x in range(NN)]

    disp_days = window + 7

    path0 = './{}/images/'.format(str_to_dt(alldates[-1]))
    if not os.path.exists(path0):
        os.makedirs(path0)

    RMSE = np.sqrt(mean_squared_error(confirm_data,confirm_all[:len(confirm_data)]))
    mape = MAPE(confirm_data,confirm_all[:len(confirm_data)])
    
    RMSE2 = np.sqrt(mean_squared_error(confirm_data,confirm_all2[:len(confirm_data)]))
    mape2 = MAPE(confirm_data,confirm_all2[:len(confirm_data)])
    
    RMSE3 = np.sqrt(mean_squared_error(confirm_data,confirm_all3[:len(confirm_data)]))
    mape3 = MAPE(confirm_data,confirm_all3[:len(confirm_data)])

    print(cityname)
    print(window)
    print("RMSE:", RMSE,RMSE2,RMSE3)
    print("MAPE:",mape,mape2,mape3)

#     plt.figure(figsize=(7,6))
#     #plt.plot(t_range_dt[1:disp_days], day_add(confirm_alln[:disp_days]), c='dimgray',linewidth=2)
#     plt.plot(t_range_dt[1:disp_days], day_add(confirm_all[:disp_days]), c='darkblue',linewidth=2)
#     plt.scatter(t_range_dt[1:len(confirm_data)], day_add(confirm_data), s=50,edgecolor='k', c='', marker='o', alpha=1,label = 'Observation')
#     plt.legend(["SI2C Simulation","Observation"], prop=legend_font,loc=7)
#     x_major_locator=MultipleLocator(7)
#     #把x轴的刻度间隔设置为7，并存在变量里
#     ax=plt.gca()
#     #ax为两条坐标轴的实例
#     ax.xaxis.set_major_locator(x_major_locator)
#     #把x轴的主刻度设置为7的倍数
    
#     plt.yticks(fontproperties = 'Times New Roman', size = 14)
#     plt.xticks(fontproperties = 'Times New Roman', size = 14)
#     #plt.gcf().autofmt_xdate()
#     plt.savefig('./{}/images/{}日新增.jpg'.format(str_to_dt(alldates[-1]),cityname,str_to_dt(alldates[-1])))
#     plt.show()
    
#     result = pd.DataFrame(data={
#         'Date': [dt_to_str(x) for x in t_range_dt[:len(confirm_data)]],
#         'SI2C_confirm': confirm_all[:len(confirm_data)],
#         'SIC_confirm':confirm_alln[:len(confirm_data)]
#     })

#     path = './{}/csv/'.format(str_to_dt(alldates[-1]))
#     if not os.path.exists(path):
#         os.makedirs(path)
#     result.to_csv(path+'{}_result_{}.csv'.format(cityname, str(dt.datetime.today().date())), index=False, encoding="gb2312")

Optimization terminated successfully.
         Current function value: 1322.856358
         Iterations: 268
         Function evaluations: 468
Optimization terminated successfully.
         Current function value: 1225.313665
         Iterations: 326
         Function evaluations: 578
Optimization terminated successfully.
         Current function value: 1384.976305
         Iterations: 276
         Function evaluations: 487
北京市
40
RMSE: 5.432692440037095 5.053326531414593 5.378261928097661
MAPE: 0.02643987925006592 0.029883447944752995 0.02986666294893614
Optimization terminated successfully.
         Current function value: 882.256602
         Iterations: 245
         Function evaluations: 432
Optimization terminated successfully.
         Current function value: 895.740292
         Iterations: 116
         Function evaluations: 213
上海市
40
RMSE: 4.972890798211616 3.6386096399433816 3.5243229976845085
MAPE: 0.04983481648088701 0.03054803135529628 0.031135905841893952
Optimization term

Optimization terminated successfully.
         Current function value: 36975.334397
         Iterations: 288
         Function evaluations: 527
荆门市
39
RMSE: 43.25767135035921 30.74080898300687 30.777989715494446
MAPE: 0.18644871772448565 0.33967917333675846 0.3514315529980689
宜昌市
37
RMSE: 31.23414285329199 22.1872399935819 22.62145631599915
MAPE: 0.20607344793962595 0.3542712592319018 0.3232665380015085
Optimization terminated successfully.
         Current function value: 7007.236579
         Iterations: 253
         Function evaluations: 437
Optimization terminated successfully.
         Current function value: 22343.990817
         Iterations: 192
         Function evaluations: 346
Optimization terminated successfully.
         Current function value: 37849.977173
         Iterations: 234
         Function evaluations: 415
黄石市
35
RMSE: 14.035206979681123 25.200819970084957 32.83381223720452
MAPE: 0.03418169506018641 0.07313277299861319 0.09926135262639699
Optimization terminated suc

E:\anaconda\Anaconda3\lib\site-packages\scipy\integrate\odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
E:\anaconda\Anaconda3\lib\site-packages\scipy\integrate\odepack.py:247: ODEintWarning: Repeated convergence failures (perhaps bad Jacobian or tolerances). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


鄂州市
36
RMSE: 741.1311403976091 144.52675700924567 268.41305640934905
MAPE: 2.8472369930587065 2.2491709595223943 3.6098283057596223
Optimization terminated successfully.
         Current function value: 81084.223422
         Iterations: 290
         Function evaluations: 502
荆州市
38
RMSE: 45.979092769532244 44.96457877810873 46.141920666616365
MAPE: 0.10425036362435683 0.10595053991200398 0.13122597654885632
